## 중앙값으로 대체하기

In [ ]:
import pandas as pd
import os
import numpy as np

out_path = r"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\interpolate"

os.chdir(r"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\C\\")
read_file = pd.read_excel("Face_1W_C1_S1.xlsx", index_col=0)

columns_to_interpolate = [
    'right eye.x', 'right eye.y', 'left eye.x', 'left eye.y',
    'nose.x', 'nose.y', 'mouse.x', 'mouse.y', 
    'right ear.x', 'right ear.y', 'left ear.x', 'left ear.y',
    'box_origin.x', 'box_origin.y', 'box.width', 'box.height'
]


# 각 컬럼의 결측치를 중앙값으로 채움
for column in columns_to_interpolate:
    median_value = read_file[column].median()
    read_file[column].fillna(median_value, inplace=True)

#print(read_file)

os.chdir(out_path)
read_file.to_excel("Face_1W_C1_S1.xlsx")


## 얼굴 기울기의 NaN 값을 보간할 수 있는 코드

In [ ]:
import os 
import pandas as pd

out_path = r"D:/MultiModal/MultiModal_Model/results/face/face_landmark/interpolate/C"
# D:\MultiModal\MultiModal_Model\results\face\face_landmark\interpolate\C

for week in ["4W"]: #4W
    for group in ["C1", "C2", "C3", "C4"]:
        # "C1", "C2", "C3", "C4", "A1", "A2", "A3", "A4"
        for step in ["S1", "S2", "S3"]:
            os.chdir(r"D:\\MultiModal\\MultiModal_Model\\results\\face\\face_landmark\\C")
            
            read_file = pd.read_excel(f"Face_{week}_{group}_{step}.xlsx", index_col=0)
            # NaN 값이 있는 곳들을 보간하는 부분.

            columns_to_interpolate = [
            'right eye.x', 'right eye.y', 'left eye.x', 'left eye.y',
            'nose.x', 'nose.y', 'mouse.x', 'mouse.y', 
            'right ear.x', 'right ear.y', 'left ear.x', 'left ear.y',
            'box_origin.x', 'box_origin.y', 'box.width', 'box.height'
            ]

            # 각 컬럼의 결측치를 중앙값으로 채움
            for column in columns_to_interpolate:
                median_value = read_file[column].median()
                read_file[column].fillna(median_value, inplace=True)
            
            # 보간된 값을 다시 엑셀로 내보내기. 
            os.chdir(out_path)
            read_file.to_excel(f"Face_{week}_{group}_{step}.xlsx")

## Face 값 중 너무 큰 값으로 튀는 outlier 제거

In [ ]:
import pandas as pd
import numpy as np
import os
#import seaborn as sns

# 데이터 표준화를 위해 scipy.stats 라이브러리 사용
import matplotlib.pyplot as plt
import scipy.stats as stats

## 이상치 제거 전, 불필요한 열을 제거하자.

In [ ]:
# file_list = ["C"] # A, D, E, F, G,
# humans = ["C1", "C2", "C3", "C4"]

#"A1", "A2", "A3", "A4"
#"B1", "B2", "B3", "B4",
#"C1", "C2", "C3", "C4",
#"D1", "D2", "D3", "D4",
#"E1", "E2", "E3", "E4",
#"F1", "F2", "F3", "F4",
#"G1", "G2", "G3", "G4"

# 불필요한 열들을 제거한 후, 최종적으로 파일들을 저장해둘 경로.
# for out in file_list:
#     for week in ["1W", "2W", "3W", "4W"]:
#         filtered_list = [human for human in humans if human.startswith(out)]
#         for human in filtered_list:
#             out_path = f"D:/MultiModal/filtered_df/{out}/" # 최종경로. 
#             # out_path = f"D:/MultiModal/filtered_df/{out}/{steps}_{human}/" # 최종경로. 

#             if not os.path.exists(out_path):
#                 os.makedirs(out_path)
                
out_path = r"D:/MultiModal/filtered_df/C/"     
                
# face 파일들이 있는 구간.
for files in ["C"]:
    default_path = f"D:/MultiModal/MultiModal_Model/results/face/face_landmark/interpolate/{files}/"
    path1 = os.listdir(default_path)
    
    # # path1 안에 들어 있는 '파일 명'만 있는 것이지 그 외는 아님. 
    for file in path1:
        if '.xlsx' in file:
            #print(file) # 파일명 출력해줌,. 
            full_path = os.path.join(default_path, file)
            #print(full_path)
            b_rotation = pd.read_excel(full_path)
            #print(b_rotation)
            filtered_df = b_rotation.drop(['right eye.x', 'right eye.y', 'left eye.x', 'left eye.y', 'nose.x', 'nose.y', 'mouse.x', 'mouse.y', 'right ear.x',
                                        'right ear.y', 'left ear.x', 'left ear.y', 'box_origin.x', 'box_origin.y'], axis = 1)
            
            filtered_df.to_excel(os.path.join(out_path, f"{file}"), index=False)

## 이상치를 제거 코드 (파일 경로 접근 + 제거)

In [ ]:
# outlier를 제거하는 부분 다시 해볼 것.

# df = 데이터 프레임 넣기, column => outlier를 제거하기 위한 열 값 할당. z_score_threshold -> 기준 값 할당.
def correct_outliers(df, column, z_score_threshold=2):
    
    mean = df[column].mean() ## 열 값에 해당하는 평균 
    std = df[column].std() ## 열 값에 해당하는 std 값 구하기 .(std* 표준편차)

    # Calculate the Z-scores
    z_scores = (df[column] - mean) / std  ## 열 값이 평균으로 부터 얼만큼 일탈되었는가 (멀어져 있는가>? )

    # z_score 값이 얼마인지 확인해볼 것. 
    print(z_scores)
    
    # Identify outliers
    # Outlier 제거하는 부분에 있어서 과하게 짤린 것 같음. =========================================================================
    outliers = abs(z_scores) > z_score_threshold ## 2보다 크면 많이 평균으로부터 많이 떨어져 있고, 매우 크거나 작다는 것을 의미. 
    #print(outliers)
    # Replace outliers with the median of the column
    median_value = df[column].median() ## 중앙값 구하기. 
    df.loc[outliers, column] = median_value ### df.loc() = median_value를 넣어줌. 

    return df

In [ ]:
import re
import math
import os
import pandas as pd

steps = ["1W", "2W", "3W", "4W"]
humans = ["C1", "C2", "C3", "C4"]

#   "A1", "A2", "A3", "A4",
#   "B1", "B2", "B3", "B4",
#   "C1", "C2", "C3", "C4",
#   "D1", "D2", "D3", "D4",
#   "E1", "E2", "E3", "E4",
#   "F1", "F2", "F3", "F4",
#   "G1", "G2", "G3", "G4"          


# load_path = "D:/MultiModal/filtered_df/"

# for group in ["A"]:   # "B", "C", "D", "E", "F", "G"
#     for step in steps:
#         filtered_human = [human for human in humans if human.startswith(group)]
#         for human in filtered_human:
#             dir 
#             if os.path.exists(load_path):
#                 os.makedirs(load_path)


for group in ["C"]:   # "B", "C", "D", "E", "F", "G"
    #for step in steps:
        #filtered_human = [human for human in humans if human.startswith(group)]
        #for human in filtered_human:

    load_path = f"D:/MultiModal/filtered_df/{group}/"
            
    # 작업 디렉토리 가져오기. 
    file_list = os.listdir(load_path)
    #print(file_list)
    
    output_folder = f"D:/MultiModal/Face_z_score/C_1/"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in file_list:
        dir = os.path.join(load_path, file)
        if dir.split('.')[-1] == "xlsx": 
            face_df = pd.read_excel(dir)

            # 뽑아내고 싶은 열 이름 모음. 
            coordinate_columns = ["box.width", "box.height"]

            for column in coordinate_columns:
                # 뽑아낼 열 이름 넣어주고, 저장. 
                filtered_df = correct_outliers(face_df, column)
                
                #print(filtered_df)
                # filtered_df 안에 값을 넣어서, 저장해줌. 
            #filtered_df.iloc[:, 1:].to_excel(output_folder + file.split('.')[0] + '.xlsx')

## 아래 코드는 내가 직접 짜본 '데이터 정규화' 코드 || 문제점 : 정규화를 하는 것은 좋은데 값이 일부 날려보냄

In [ ]:
groups = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
steps = ['1W', '2W', '3W', '4W']
humans = ['A1', 'A2', 'A3', 'A4',
          'B1', 'B2', 'B3', 'B4',
          'C1', 'C2', 'C3', 'C4',
          'D1', 'D2', 'D3', 'D4',
          'E1', 'E2', 'E3', 'E4',
          'F1', 'F2', 'F3', 'F4',
          'G1', 'G2', 'G3', 'G4']

# 저장할 수 있는 경로 설정.
for group in groups:
    for step in steps:
        for human in humans:
            result_out_path = f"D:/MultiModal/Face_z_score/{group}/{step}_{human}/"
            if not os.path.exists(result_out_path):
                os.makedirs(result_out_path)

# 경로 설정 
for group in groups:
    for step in steps:
        for human in humans:
            face_file_path = f"D:/MultiModal/filtered_df/{group}/{step}_{human}/"
            
            if os.path.exists(face_file_path):
                files = os.listdir(face_file_path)
                # 엑셀 파일들 리스트 출력하기
                #print(files)     
                
                # files -> listdir()를 통해서 list 형태로 있는 값들을 불러온 것. 
                for i in files:
                    full_path1 = os.path.join(face_file_path, i) # 경로에 있는 값 하나씩 불러오기. 
                    temp_file = pd.read_excel(full_path1) # 하나씩 불러왔으니, 이를 읽기 위해 read_excel 사용. 
                    #print(temp_file)
            
                    # 이상치를 구하기 위한 식) : (값 - 평균) / (표준편차)  => z_score 값
                    outlier_face = temp_file[(abs((temp_file['box.height'] - np.mean(temp_file['box.height'])) / np.std(temp_file['box.height']))) > 1.96].index
                    # 이상치로 판단되는 값을 제거. 
                    new_df = temp_file.drop(outlier_face)
                    #print(new_df)
                    
                    #이상치를 제거했다면, 결과 값을 엑셀에 저장. 
                    new_df.to_excel(os.path.join(result_out_path, f"filtered_outlier_{i}"), index= False)

## 델타 값 계산해서, 그래프 그리기.

In [ ]:
import seaborn as sns

# body_lean_delta 값 계산
body_path = "D:\\MultiModal\\body_lean_all\\F\\"
# B,C,D,E,F,G

# face_lean_delta 값 계산
face_path = "D:/MultiModal/filtered_df/F/"
# B,C,D,E,F,G

out_path = "D:/MultiModal/data_visualize/F/"
if not os.path.exists(out_path):
    os.makedirs(out_path) 

# 각각 뽑아봐야 하는 것들. 
steps = ['4W'] # '2W', '3W', '4W'
humans = ['F1']
# 'A2', 'A3', 'A4'

for step in steps:
    for human in humans:
        dir = os.path.join(face_path, f"{step}_{human}")
        file_list = os.listdir(dir)
        
        dir_body = os.path.join(body_path, f"{step}_{human}")
        file_body_list = os.listdir(dir_body)
        
        for file_b, files in zip(file_body_list, file_list):
            file_zip_b = os.path.join(dir_body, file_b)
            file_dir_b = pd.read_excel(file_zip_b)
            
            # body에서의 각 델타 값을 계산. 
            filtered_delta_b = file_dir_b.diff()
            
            # 그래프를 그리긱 전 NaN 값을 제거. *보간. 
            filtered_delta_b.interpolate(inplace=True)

            # 차이 값이 계산되는지 확인. 
            #print(filtered_delta_b)
            
            file_zip = os.path.join(dir, files)
            read_dir = pd.read_excel(file_zip)
            
            # 불필요한 열 제거. 
            read_dir.drop(['Unnamed: 0'], axis=1, inplace= True)
            
            # 각 델타 값들을 계산
            filtered_delta = read_dir.diff()
            print(filtered_delta) # 비교할 값이 없다면, NaN 값으로 출력. 

            # 그래프를 그리긱 전 NaN 값을 제거. 
            filtered_delta.interpolate(inplace=True)
        
            # 두 개의 데이터 프레임의 최소값을 맞춰주는 것.     
            min_length = min(len(filtered_delta), len(filtered_delta_b))
            filtered_delta = filtered_delta[:min_length]
            filtered_delta_b = filtered_delta_b[:min_length]
    
            # 산점도를 확인해볼 수 있는 것. 
            #lt.scatter(filtered_delta, filtered_delta_b)
            #plt.xlabel('body_lean')  # 바디 기울기.
            #plt.ylabel('face_lean')  # 얼굴 기울기. 
            #plt.title('Scatter Plot of face_diff() vs. body_lean_diff()')
            #plt.grid(True)  # Enable grid
            #plt.show()
        
            new_df =  pd.concat([filtered_delta, filtered_delta_b], axis=1)
            new_df.columns = ['filtered_delta', 'filtered_delta_b']
            
            # 뽑아보고 싶은 특정 열 선택. 
            data_range = new_df.loc[900:1000]
            # Calculate the correlation matrix
            correlation_matrix = new_df.corr()
            plt.figure(figsize=(10, 6))
            
            plt.plot(data_range['filtered_delta'], label='filtered_delta')
            plt.plot(data_range['filtered_delta_b'], label='filtered_delta_b')
            plt.xlabel('Index')
            plt.ylabel('Values')
            plt.title('Graph of face box and body_lean (Index Range: 5100-5450)')
            plt.legend()
            plt.grid(True)
            plt.show()
                    
        
        
            # 히트 맵으로 보여주는 코드 부분
            # 양의 상관 관계는 따뜻한 색상, 음의 상관관계는 파란색으로 나타남.
            # 값이 1또는 -1에 가까울 수록 강한 상관관계를 나타내고, 0에 가까울수록 약한 상관 관계를 나타냄. 
            # Plotting heatmap of correlation matrix
            # plt.figure(figsize=(8, 6))
            # sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
            # plt.title('Correlation Heatmap between filtered_delta and filtered_delta_b')
            # plt.show()
    

## face box scaling 해주는 구간.

In [ ]:
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

for group in ["A"]: #A
    os.chdir(f"D:/MultiModal/Face_z_score/{group}/") 
    
    ## D:\MultiModal\MultiModal_postprocessing\results\face\face_landmark\G
    
    # 파일 불러오기
    file_list = os.listdir(os.getcwd())

    # 출력할 파일 불러오기
    output_folder = f"D:/MultiModal/Face_z_score/Face_lean/{group}/"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 컬럼명
    face_landmark_col = ["Week", "ID", "Step", "Face_lean_width", "Face_lean_height"]

    data = []
    for file in file_list:
        if file.split('.')[-1] == "xlsx":
            landmark_df = pd.read_excel(file)
            # scaling
            landmark_df["lean_width"] = min_max_scaling(landmark_df["box.width"]) # 너비
            landmark_df["lean_height"] = min_max_scaling(landmark_df["box.height"]) # 높이
            
            file_name = file.split('.')[0].split('_')
            ## + '_' + file_name[-1]
            data.append([file_name[1], file_name[2], file_name[3] if (file_name[3] == "Baseline" or file_name[3] == "S3") else file_name[3], np.mean(landmark_df["lean_width"]), np.mean(landmark_df["lean_height"])])

    avg_box_df = pd.DataFrame(data, columns=face_landmark_col)
    avg_box_df.to_excel(output_folder + f"{group}_average_face_lean.xlsx")

## 평균 Face lean 값을 정리하여 뽑아보는 구간

In [ ]:
# all df concat

group_df = []

for group in ["A", "B", "C", "D", "E", "F", "G"]:
    os.chdir(f"D:/MultiModal/Face_z_score/Face_lean/{group}/") 
    ## D:\MultiModal\MultiModal_postprocessing\results\face_time\face_landmark\G
    group_df.append(pd.read_excel(f"{group}_average_face_lean.xlsx").iloc[:, 1:])

    output_folder = f"../"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

all_group_df = pd.concat(group_df)
all_group_df.to_excel(output_folder + "All_average_face_lean(All group, anaysis).xlsx")

## 데이터 분석용으로 출력하는 파일

In [ ]:
os.chdir("D:/MultiModal/Face_z_score/Face_lean/") 
all_df = pd.read_excel("All_average_face_lean(All group, anaysis).xlsx") 

step_list = ["S1", "S2" "S3"]

df_index = ["Group", "ID"] + [f"{week}_{step}_Face_lean_{option}" for week in ["1W", "2W", "3W", "4W"] for step in step_list for option in ["width", "height"]] 

data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    for id in [f"{group}{id_index}" for id_index in range(1, 5)]:
        week_data = []
        for week in [f"{w_index}W" for w_index in range(1, 5)]:
            filtered_df = all_df[(all_df["Week"] == week) & (all_df["ID"] == id) & (all_df["Group"] == group)]
            print(filtered_df)
            week_data.extend(filtered_df[[f"{step}_Face_lean_{option}" for step in step_list for option in ["width", "height"]]].values.flatten().tolist())
        
        data.append([group, id] + week_data)

# all_summarize_df = pd.DataFrame(data, columns=df_index)
# all_summarize_df.to_excel("All_average_face_lean(All group, excel_anaysis).xlsx")

## 얼굴 기울기 횟수 세는 구간.

In [ ]:
# 얼굴 크기 변화량 값 계산하는 구간. 
def plus_minus(df):
    df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )

    W = "box.width"
    H = "box.height"

    df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
    df_new = df.iloc[1:, :]["box delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 
    
    # df_new 값일 어떻게 생성되는지 확인해보기 위해 출력. 
    #print(df_new)
    
    for i in range(len(df_new)):
        
        if(df_new.iloc[i] > 600):  # 평균적인 사이즈 값
            df_new.iloc[i] = 1 # 앞으로 기울었을 때 
        elif(df_new.iloc[i] < -600):
            df_new.iloc[i] = -1 # 뒤로 기울었을 때 
        else:
            df_new.iloc[i] = 0 # 아무것도 아닐 때
    return df_new 

In [ ]:
import os
import pandas as pd
import numpy as np

# 최종적으로 저장할 경로. 
data = []

output_folder = f"D:/MultiModal/Face_z_score/Face_lean_count/A/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for group in ["A"]: # , "B","C", "D", "E", "F", "G"
    os.chdir(f"D:/MultiModal/Face_z_score/{group}/") 

    # 스텝에 나와 있는 것 
    step_list = ["S1", "S2", "S3"]
    #step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

    face_index = ["Group", "Week",] + [f"{step}_Face_lean_4p" for step in step_list] + [f"{step}_Face_lean_3p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        
        step_data_4p = []
        step_data = []
        for step in step_list:
            df1 = pd.read_excel(f'Face_{week}_{group}1_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}1_{step}.xlsx') else []
            df2 = pd.read_excel(f'Face_{week}_{group}2_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}2_{step}.xlsx') else []
            df3 = pd.read_excel(f'Face_{week}_{group}3_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}3_{step}.xlsx') else []
            df4 = pd.read_excel(f'Face_{week}_{group}4_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}4_{step}.xlsx') else []
            
            #이상이 따로 없다면, df1~4까지의 len() 길이 값을 받아오고, 데이터 프레임이 비어있는 경우에는 100000000 값을 넣어주어 대체함. 
            a = np.array([len(df1) if len(df1) > 0 else 100000, len(df2) if len(df2) > 0 else 100000, len(df3) if len(df3) > 0 else 100000, len(df4) if len(df4) > 0 else 100000])
            #length = a.min()
            length = a.min()-1 # 최솟값에서 -1을 해줌. 

            # # def문을 활용해서 delta 값 만들기. 
            # def plus_minus(df):
            #     df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )
 
            #     W = "box.width"
            #     H = "box.height"

            #     df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
            #     df_new = df.iloc[1:, 18] # ? 왜 열 값을 이렇게 주었을까? 첫 1번 열과 18개의 행만  
                
            #     for i in range(len(df_new)):
            #         if(df_new.iloc[i] > 600):  # 평균적인 사이즈 값
            #             df_new.iloc[i] = 1 # 앞으로 기울었을 때 
            #         elif(df_new.iloc[i] < -600):
            #             df_new.iloc[i] = -1 # 뒤로 기울었을 때 
            #         else:
            #             df_new.iloc[i] = 0 # 아무것도 아닐 때
            #     return df_new 
   
            # df1 = plus_minus(df1)
            # print(df1)
   
   
            if len(df1) > 0:
                df1 = plus_minus(df1).iloc[:length] # if length > 0 else None
            else: 
                df1 = pd.DataFrame({"box delta_A1": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            if len(df2) > 0:
                df2 = plus_minus(df2).iloc[:length] 
            else:
                df2 = pd.DataFrame({"box delta_A2": [0 for _ in range(length)]}) #{"box delta": [0 for _ in range(length)]}

            if len(df3) > 0:
                df3 = plus_minus(df3).iloc[:length]  
            else:
                df3 = pd.DataFrame({"box delta_A3": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}

            if len(df4) > 0:
                df4 = plus_minus(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"box delta_A4": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
            
            # 각 그룹별 데이터가 저장되어 있는 데이터 프레임 값 저장. || axis = 1 (세로 (열) 형태로 저장될 수 있도록 함. ) 
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)
            #print(df_concat.head())

            df_concat['same'] = 0 
                        
            for i in range (len(df_concat)):
                
                count=df_concat.iloc[i].value_counts()
                try:
                    count[0] = 0
                except:
                    0
            
                df_concat.iloc[i,4] = count.max()
            
            same = np.array(df_concat["same"])
            #step_data.append((df_concat['same'] >= 3).sum())
            
            # 4명 모두 기울인 것.
            step_data_4p.append(np.where(same>=4,1,0).sum())
        
        data.append([group, week] + step_data_4p)
        # 새로운 데이터 프레임을 하나 만들어줄 것. 
        count_df = pd.DataFrame(data, columns=face_index)
    
         # out_folder 위치에 face_lean이 얼마나 기울어졌는지 기록하는 구간.  
        count_df.to_excel(output_folder + "Face_lean_count.xlsx")

In [ ]:
# 아래의 경로로 변경
os.chdir("D:/MultiModal/Face_z_score/Face_lean_count/")
all_df = pd.read_excel("C_Face_lean_count.xlsx") 

#step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

step_list = ["S1", "S2", "S3"]

df_index = ["Group"] + [f"{week}_{step}_Face_lean_3p" for week in ["1W", "2W", "3W", "4W"] for step in step_list] 
### , "2W", "3W", "4W"
data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    week_data = []
    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        filtered_df = all_df[(all_df["Week"] == week) & (all_df["Group"] == group)]
        week_data.extend(filtered_df[[f"{step}_Face_lean_3p" for step in step_list]].values.flatten().tolist())
        print(filtered_df)
    
    data.append([group] + week_data)   

all_summarize_df = pd.DataFrame(data, columns=df_index)
all_summarize_df.to_excel("All_Face_lean_count(All group, excel_anaysis).xlsx")

## 몸의 기울기 값 z_score 이상치 제거 코드.

In [ ]:
def correct_outliers(df, column, z_score_threshold=3):
    
    mean = df[column].mean()
    std = df[column].std()

    z_scores = (df[column] - mean) / std
    
    outliers = abs(z_scores) > z_score_threshold

    median_value = df[column].median()
    df.loc[outliers, column] = median_value

    return df

In [ ]:
"""
Body Scaling (이상치 처리) 정리 || 값을 조정해주는 코드
"""
import re
import math

for group in ["A", "B", "C", "D", "E", "F", "G"]:
    os.chdir(f"Users/nabi/Desktop/filtered_df/{group}/")
    
    # 파일 불러오기
    file_list = os.listdir(os.getcwd())
    # 출력할 파일 불러오기
    output_folder = f"Users/nabi/Desktop/filtered_df_result/"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in file_list:
        if file.split('.')[-1] == "xlsx": # xlsx 파일만 읽기
            body_df = pd.read_excel(file)

            coordinate_columns = [col for col in body_df.columns if col.endswith(('.x', '.y', '.z'))]

            for column in coordinate_columns:
                filterd_df = correct_outliers(body_df, column)

            filterd_df.iloc[:, 1:].to_excel(output_folder + file)

## 몸의 기울기 값도 똑같이 얼굴 기울기를 셀 수 있도록 해보자. 

In [ ]:
def plus_minus_1(df):
 
    # body delta라는 이름의 데이터 프레임에 lambda로 조건식 주는 곳
    #df["body delta"] = df["body_lean"].apply(lambda x: 1 if x < 90 else (-1 if x > 90 else 0))
    
    df["body delta"] = df["Body_lean"].diff()
    #print(df["body delta"])
    
    # 첫 번째 열은 비교할 수 있는 값이 없으므로, NaN값이 생길 수 있음. 이를 방지하기 위해 0 값을 할당. 
    df.loc[0, 'Body_lean'] = 0
    df_new = df.iloc[1:, :]["body delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 

    # 절대값을 계산해서 비교.
    #df_diff = abs(df_new)
    
    for i in range(len(df_new)):
        
        if(df_new.iloc[i] > 0.02):  # 평균적인 사이즈 값
            df_new.iloc[i] = 1 # 앞으로 기울었을 때 
        elif(df_new.iloc[i] < -0.02):
            df_new.iloc[i] = -1 # 뒤로 기울었을 때 
        elif(-0.02 < df_new.iloc[i] < 0.02):
            df_new.iloc[i] = 0 # 아무것도 아닐 때
    
    return df_new 

In [ ]:
import os
import pandas as pd
import numpy as np

# 최종적으로 저장할 경로. 
data = []

# 출력할 수 있는 경로 설정 "D:/MultiModal/body_lean_all/body_lean_count/"
output_folder = f"/Users/nabi/Desktop/body_lean_count/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for group in ["A","B","C","D","E","F","G"]: # f"D:/MultiModal/body_lean_all/{group}/"
    os.chdir(f"/Users/nabi/Desktop/filtered_mean_value/{group}/")
            
    #스텝에 나와 있는 것 
    step_list = ["S1", "S2", "S3"]
    body_index = ["Group", "Week",] + [f"{step}_Body_lean_3p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        step_data = []
        for step in step_list: # lean_
            df1 = pd.read_excel(f'Mean_filtered_{week}_{group}1_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}1_{step}.xlsx') else []
            df2 = pd.read_excel(f'Mean_filtered_{week}_{group}2_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}2_{step}.xlsx') else []
            df3 = pd.read_excel(f'Mean_filtered_{week}_{group}3_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}3_{step}.xlsx') else []
            df4 = pd.read_excel(f'Mean_filtered_{week}_{group}4_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}4_{step}.xlsx') else []
            
            #print(df1)
            #print(df2)
            #print(df3)
            #print(df4)
            
            #이상이 따로 없다면, df1~4까지의 len() 길이 값을 받아오고, 데이터 프레임이 비어있는 경우에는 100000000 값을 넣어주어 대체함. 
            a = np.array([len(df1) if len(df1) > 0 else 100000, len(df2) if len(df2) > 0 else 100000, len(df3) if len(df3) > 0 else 100000, len(df4) if len(df4) > 0 else 100000])
            
            length = a.min()-1 # 최솟값에서 -1을 해줌. 

            if len(df1) > 0:
                df1 = plus_minus_1(df1).iloc[:length] # if length > 0 else None
            else: 
                df1 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            if len(df2) > 0:
                df2 = plus_minus_1(df2).iloc[:length] 
            else:
                df2 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) #{"box delta": [0 for _ in range(length)]}

            if len(df3) > 0:
                df3 = plus_minus_1(df3).iloc[:length]  
            else:
                df3 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}

            if len(df4) > 0:
                df4 = plus_minus_1(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            # 각 그룹별 데이터가 저장되어 있는 데이터 프레임 값 저장. || axis = å1 (세로 (열) 형태로 저장될 수 있도록 함. ) 
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)
            #print(df_concat.head())

            # same 열 초기화 
            df_concat['same'] = 0 
                        
            for i in range (len(df_concat)):
                #print(i)
                # df_concat.iloc[] 번째의 값을 카운트
                count=df_concat.iloc[i].value_counts()
                try:
                    count[0] = 0
                except:
                    0
            
                df_concat.iloc[i,4] = count.max()
            
            same = np.array(df_concat["same"])  
            # step_data 리스트 안에는 same 값이 3보다 크다면 인덱스 값을 반환해서, 만족하면 1 아니면 0을 반환. 
            step_data.append(np.where(same>=3,1,0).sum())
    
        data.append([group, week] + step_data)
        # # 새로운 데이터 프레임을 하나 만들어줄 것. 
        count_df = pd.DataFrame(data, columns=body_index)
    
        # out_folder 위치에 face_lean이 얼마나 기울어졌는지 기록하는 구간.  
        count_df.to_excel(output_folder + "All_Body_lean_count.xlsx")

In [ ]:
# 아래의 경로로 변경
os.chdir("/Users/nabi/Desktop/body_lean_count/") # "D:/MultiModal/body_lean_all/body_lean_count/"
all_df = pd.read_excel("All_Body_lean_count.xlsx") 

#step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

step_list = ["S1", "S2", "S3"]

df_index = ["Group"] + [f"{week}_{step}_Body_lean_3p" for week in ["1W", "2W", "3W", "4W"] for step in step_list] 
### , "2W", "3W", "4W"
data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    week_data = []
    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        filtered_df = all_df[(all_df["Week"] == week) & (all_df["Group"] == group)]
        week_data.extend(filtered_df[[f"{step}_Body_lean_3p" for step in step_list]].values.flatten().tolist())
    
    data.append([group] + week_data)   

all_summarize_df = pd.DataFrame(data, columns=df_index)
all_summarize_df.to_excel("All_Body_lean_count(All group, excel_anaysis).xlsx")

## body_lean의 diff() 값을 계산해보자. 

In [ ]:
import numpy as np

def show_the_diff(df):
    # body_diff 라는 이름의 열 하나 생성. || 그리고 body_lean에 대한 diff() * 열 간의 차이 값 계산. 
    df['body diff'] = df['Body_lean'].diff()
    
    # 첫 번째 열은 비교할 수 있는 값이 없으므로, NaN값이 생길 수 있음. 이를 방지하기 위해 0 값을 할당. 
    df.loc[0, 'Body_lean'] = 0
        
    # diff_df 라는 변수 안에 body diff 전체 값을 넣어 줌. 
    diff_df = df.iloc[1:, :]["body diff"]
    
    result_df = np.median(diff_df)
    result = abs(result_df)

    # 마지막 result에는 diff() 값을 반환. 
    return result

In [ ]:
import os
import pandas as pd
# 최종적으로 저장할 경로. 
data = []

for group in ["B"]: 
    os.chdir(f"/Users/nabi/Desktop/filtered_mean_value/{group}/")

    read_dir = pd.read_excel("Mean_filtered_4W_B1_S1.xlsx")
    result_diff = show_the_diff(read_dir)
    
    print(result_diff)

## 인덱스 정보 값을 넣어주기.

In [ ]:
import os
import pandas as pd

body_path = r"D:\\MultiModal\\body_lean_all\\G\\"
 
outpath = r"D:\\MultiModal\\body_lean_all\\body_lean_count\\lean(4p_2)\\G\\"
 
 
# 파일 리스트 값을 불러오기. 
list = os.listdir(body_path)

# 리스트 내에 있는 파일 하나씩 읽어오기. 
for file in list:
    read_file = os.path.join(body_path, file)
    read_f = pd.read_excel(read_file)
    
    index_val = []
    
    for i in read_f.index:
        index_val.append(i)
        
    new_df = pd.DataFrame(columns= ["Frame"])
    new_df["Frame"] = index_val
    
    result = pd.concat([new_df, read_f], axis= 1, ignore_index=False)
    result.style.hide(axis='index')
    
    result.to_excel(outpath + f"Add_{file}", index=False)
    #print(result)

## 4명 중에 N명 기울어진 정도 + Frame 시간에 맞게 뽑아낸 것


In [ ]:
# 얼굴 크기 변화량 값 계산하는 구간. 
def plus_minus(df):
    df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )

    W = "box.width"
    H = "box.height"

    df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
    df_new = df.iloc[1:, :]["box delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 
    # df_new 값일 어떻게 생성되는지 확인해보기 위해 출력. 
    #print(df_new)
    
    for i in range(len(df_new)):
        
        if(df_new.iloc[i] > 250):  # 평균적인 사이즈 값
            df_new.iloc[i] = 1 # 앞으로 기울었을 때 
        elif(df_new.iloc[i] < -250):
            df_new.iloc[i] = -1 # 뒤로 기울었을 때 
        else:
            df_new.iloc[i] = 0 # 아무것도 아닐 때
    return df_new 

In [ ]:
import os
import pandas as pd
import numpy as np

# 최종적으로 저장할 경로. 
data = []

# 5분 단위로 자른 것이 아닐 때의 값을 저장하기 위한 부분. 
# output_folder = f"D:/MultiModal/Face_z_score/Face_lean_count/backup/box_delta_not_5m/"

output_folder = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\C_250\\"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# D:\MultiModal\Face_z_score\Face_lean_count\backup\All_face_box_zip\box_delta_not_5min

for group in ["C"]: # "A","B","C", "D", "E", "F", "G"
    os.chdir(f"D:/MultiModal/Face_z_score/{group}/") 
    
    # 5분 단위로 잘라 놓은 부분들의 diff() 값들을 구해 놓은 부분. 
    #os.chdir(f"D:/MultiModal/Face_z_score/face_landmark_Split/{group}/") 
    step_list = ["S1", "S2", "S3"]
    #step_list = ["S1_1", "S1_2", "S1_3","S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

    # 3명이 기울었을 때, 2명이 기울었을 때, 1명이 기울었을 때.
    #face_index = ["Group", "Week",] + [f"{step}_Face_lean_3p" for step in step_list] + [f"{step}_Face_lean_2p" for step in step_list] + [f"{step}_Face_lean_1p" for step in step_list]
    face_index = [f"{step}_Face_lean_3p" for step in step_list] + [f"{step}_Face_lean_2p" for step in step_list] + [f"{step}_Face_lean_1p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        # step 3개.
        for step in step_list:
            df1 = pd.read_excel(f'Face_{week}_{group}1_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}1_{step}.xlsx') else []
            df2 = pd.read_excel(f'Face_{week}_{group}2_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}2_{step}.xlsx') else []
            df3 = pd.read_excel(f'Face_{week}_{group}3_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}3_{step}.xlsx') else []
            df4 = pd.read_excel(f'Face_{week}_{group}4_{step}.xlsx') if os.path.exists(f'Face_{week}_{group}4_{step}.xlsx') else []
            
            #이상이 따로 없다면, df1~4까지의 len() 길이 값을 받아오고, 데이터 프레임이 비어있는 경우에는 100000 값을 넣어주어 대체함. 
            a = np.array([len(df1) if len(df1) > 0 else 100000, len(df2) if len(df2) > 0 else 100000, len(df3) if len(df3) > 0 else 100000, len(df4) if len(df4) > 0 else 100000])
        
            length = a.min()-1 # 최솟값에서 -1을 해줌. 
            
            if len(df1) > 0:
                df1 = plus_minus(df1).iloc[:length] # if length > 0 else None
            else: 
                df1 = pd.DataFrame({"box delta_C1": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            if len(df2) > 0:
                df2 = plus_minus(df2).iloc[:length] 
            else:
                df2 = pd.DataFrame({"box delta_C2": [0 for _ in range(length)]}) #{"box delta": [0 for _ in range(length)]}

            if len(df3) > 0:
                df3 = plus_minus(df3).iloc[:length]  
            else:
                df3 = pd.DataFrame({"box delta_C3": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}

            if len(df4) > 0:
                df4 = plus_minus(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"box delta_C4": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
            
            # 각 그룹별 데이터가 저장되어 있는 데이터 프레임 값 저장. || axis = 1 (세로 (열) 형태로 저장될 수 있도록 함. ) 
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)
            
            df_concat.to_excel(output_folder + f"box_delta_diff_{week}_{group}_{step}.xlsx")

## box_delta 1~4 중 각 조건에 맞게 12프레임 중 8개가 -1 이라면 -1로 쭉 기록 || 1이라면 1로 쭉 기록

## -1 혹은 1이 10개 이상 되는 행을 찾아보자

In [ ]:
def find_consecutive_values(series, value, min_count):
    count = 0
    start_index = -1
    ranges = []
    
    for i, v in enumerate(series):
        if v == value:
            if count == 0:
                start_index = i
            count += 1
        else:
            if count >= min_count:
                ranges.append((start_index, i - 1))
            count = 0
    
    if count >= min_count:
        ranges.append((start_index, len(series) - 1))
    
    return ranges

## 첫 행부터 50행(=25 프레임)까지, 1 혹은 -1의 값이 얼마나 있을까

In [ ]:
def sliding_window_count(df, window_size, value_list, min_count):
    n = len(df)
    for start in range(n - window_size + 1):
        end = start + window_size
        subset_df = df.iloc[start:end]
        count = subset_df['box delta.2'].isin(value_list).sum()
        if count >= min_count:
            return start, end - 1, count
    return None

## 1이 0~50행 내에서 20개 이상 있는지 확인하기 위한 것

In [5]:
def sliding_window_count_ones(df, window_size, target_value, min_count): # window_size => 50개의 행만큼 건너뛰어서 보고 싶은 것. 
    # target_value = 1(or 1) # min_count = 20개(* 80%)
    n = len(df)
    list1 = []
    for start in range(0, n, window_size): # window_size 만큼 건너뛰면서 확인. 
        end = start + window_size
        subset_df = df.iloc[start:end]
        count = (subset_df['box delta.1'] == target_value).sum() # lean_3p
        # count >=12 이라면, 갯수가 몇 개인 지를 카운트 해주는 부분
        if count >= min_count:
            list1.append((start, end - 1, count))     
            
    return list1

In [17]:
def sliding_window_count_ones(df, window_size, target_value, min_count):
    n = len(df)
    columns_to_check = ['box delta', 'box delta.1', 'box delta.2', 'box delta.3']
    
    for start in range(0, n, window_size):
        end = min(start + window_size, n)  # Ensure we do not go out of bounds
        subset_df = df.iloc[start:end]
        
        # Check if any of the columns in the current window meet the condition
        for column in columns_to_check:
            count = (subset_df[column] == target_value).sum()
            if count >= min_count:
                df.iloc[start:end, df.columns.get_indexer(columns_to_check)] = target_value
                break  # Once one column meets the condition, we update the entire window and move on
                
    return df

In [12]:
import os 
import pandas as pd

## box.delta 값이 연속으로 1인 값이 7개 이상일 때의 열을 찾는 코드

In [ ]:
path = r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/05_27_box_delta_zip/C_250/"
# D:\MultiModal\Face_z_score\Face_lean_count\backup\All_face_box_zip\05_27_box_delta_zip\C_250
#print(os.getcwd())

os.chdir(path)
#print(os.getcwd())
for week in ["1W", "2W", "3W", "4W"]:
    for step in ["S1", "S2", "S3"]:
        read_data = pd.read_excel(f"box_delta_diff_{week}_C_{step}.xlsx", index_col=0)
        
        # -1 값이 7개 이상으로 나오는 곳 
        consecutive_ranges = find_consecutive_values(read_data['box delta.1'], 1, 7)
        for start, end in consecutive_ranges:
            print(f"연속된 1 값이 {week}_C_{step}파일에서, {start}에서 {end}까지 발견됨")

## 0~50행까지 계속 탐색해보았을 떄, 1이 연속으로 20개 이상 있는 곳을 찾는 코드

In [8]:
def sliding_window_count_values(df, window_size, target_value_1, target_value_2, min_count):
    n = len(df)
    columns_to_check = ['box delta', 'box delta.1', 'box delta.2', 'box delta.3']
    
    for start in range(0, n, window_size):
        end = min(start + window_size, n)  # Ensure we do not go out of bounds
        subset_df = df.iloc[start:end]
        
        # Check counts for both target values
        count_1 = (subset_df[columns_to_check] == target_value_1).sum().sum()
        count_minus_1 = (subset_df[columns_to_check] == target_value_2).sum().sum()
        
        if count_1 >= min_count:
            df.iloc[start:end, df.columns.get_indexer(columns_to_check)] = target_value_1
        elif count_minus_1 >= min_count:
            df.iloc[start:end, df.columns.get_indexer(columns_to_check)] = target_value_2
                
    return df

In [7]:
#path = r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/05_27_box_delta_zip/C_250/"
# D:\MultiModal\Face_z_score\Face_lean_count\backup\All_face_box_zip\05_27_box_delta_zip\C_250
input = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Check\\"
#print(os.getcwd())

os.chdir(input)
#print(os.getcwd())
for week in ["1W", "2W", "3W", "4W"]:
    for step in ["S1", "S2", "S3"]:
        read_data = pd.read_excel(f"box_delta_{week}_C_{step}.xlsx", index_col=0)
        
        # 첫 50행씩 이동하며 1 또는 -1의 개수를 확인
        #result = sliding_window_count(read_data, 50, [-1, 1], 20)
        result = sliding_window_count_ones(read_data, 12, -1, 8)    
        # 결과 출력
        if result:
            for start, end, count in result:
                print(f"{week}_C_{step}파일에서, 행 {start}에서 {end}까지의 구간에서 값이 1인 개수는 {count}입니다.")
        else:
            print("조건을 만족하는 구간이 없습니다.")

1W_C_S1파일에서, 행 2292에서 2303까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 2460에서 2471까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 3624에서 3635까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 6540에서 6551까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 8232에서 8243까지의 구간에서 값이 1인 개수는 9입니다.
1W_C_S1파일에서, 행 10104에서 10115까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 11664에서 11675까지의 구간에서 값이 1인 개수는 9입니다.
1W_C_S1파일에서, 행 14724에서 14735까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 14748에서 14759까지의 구간에서 값이 1인 개수는 9입니다.
1W_C_S1파일에서, 행 14760에서 14771까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 16236에서 16247까지의 구간에서 값이 1인 개수는 9입니다.
1W_C_S1파일에서, 행 16356에서 16367까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 26496에서 26507까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 27540에서 27551까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S1파일에서, 행 29496에서 29507까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S2파일에서, 행 2880에서 2891까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S2파일에서, 행 4080에서 4091까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S2파일에서, 행 4164에서 4175까지의 구간에서 값이 1인 개수는 9입니다.
1W_C_S2파일에서, 행 5520에서 5531까지의 구간에서 값이 1인 개수는 8입니다.
1W_C_S2파일에서

In [9]:
import os
import pandas as pd
path = r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/05_27_box_delta_zip/C_250/"
if not os.path.exists(path):os.makedirs(path)
# input = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Check\\"
#print(os.getcwd())

os.chdir(path)
#print(os.getcwd())
for week in ["1W", "2W", "3W", "4W"]:
    for step in ["S1", "S2", "S3"]:
        read_data = pd.read_excel(f"box_delta_diff_{week}_C_{step}.xlsx", index_col=0)
        # box_delta_diff_1W_C_S1
        
        # 첫번째부터 12행씩 이동하며 1 또는 -1의 개수를 확인
        result = sliding_window_count_values(read_data, 12, 1, -1, 8)    
        # 결과 출력
        result_path = f"modified_box_delta_{week}_C_{step}.xlsx"
        result.to_excel(result_path)
        print(f"{result_path} 파일이 저장되었습니다.")

modified_box_delta_1W_C_S1.xlsx 파일이 저장되었습니다.
modified_box_delta_1W_C_S2.xlsx 파일이 저장되었습니다.
modified_box_delta_1W_C_S3.xlsx 파일이 저장되었습니다.
modified_box_delta_2W_C_S1.xlsx 파일이 저장되었습니다.
modified_box_delta_2W_C_S2.xlsx 파일이 저장되었습니다.
modified_box_delta_2W_C_S3.xlsx 파일이 저장되었습니다.
modified_box_delta_3W_C_S1.xlsx 파일이 저장되었습니다.
modified_box_delta_3W_C_S2.xlsx 파일이 저장되었습니다.
modified_box_delta_3W_C_S3.xlsx 파일이 저장되었습니다.
modified_box_delta_4W_C_S1.xlsx 파일이 저장되었습니다.
modified_box_delta_4W_C_S2.xlsx 파일이 저장되었습니다.
modified_box_delta_4W_C_S3.xlsx 파일이 저장되었습니다.


## Box delta 간의 diff() 값을 구했으니, 이에 맞게 몇 명이 기울었는지를 체크하는 구간.

In [20]:
import os
import pandas as pd 
import numpy as np

data = []

### 5분 단위로 자르지 않은 부분들을 모아놓은 곳. 
#out_folder = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\Output_C\\"
Out_folder = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Check_C250\\"
if not os.path.exists(Out_folder):
    os.makedirs(Out_folder)

#input = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\C\\"
#input = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\box_delta_not_5min\\"
path = r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/05_27_box_delta_zip/C_250/"


for group in ["C"]: # "C"
    for week in ["1W","2W","3W","4W"]:
        for step in ["S1", "S2", "S3"]: # , "S2", "S3"
            # "S1_1", "S1_2", "S1_3","S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"
            os.chdir(path)
            read_file = pd.read_excel(f"modified_box_delta_{week}_{group}_{step}.xlsx")
            #print(len(read_file))
            
            step_data_3p = []
            step_data_2p = []
            step_data_1p = []
            
            # 동시에 기울어진 사람들을 카운트 하는 구간. 
            read_file['same']=0 
                    
            for i in range (len(read_file)):
                #print(read_file.iloc[i, 1:5])
                val_count = read_file.iloc[i,1:5].value_counts()
                # val_count 안에 있는 값이 0 이라면 0으로 초기화. 
                val_count[0] = 0
                # 최댓값을 넣어주려는 부분. 
                #print(read_file.iloc[i, :5])
                read_file.iloc[i,5] = val_count.max()
                
            same = np.array(read_file["same"])

            # 동시에 4명 중 N명이 기울었을 때를 세는 부분 || 모두 list 안에 들어있는 것. 
            step_data_3p = np.where(same>=3,1,0)
            step_data_2p = np.where(same>=2,1,0)
            step_data_1p = np.where(same>=1,1,0)
        
            # lean_4p, 3p,2p,1p에 맞게 데이터 값을 데이터 프레임으로 지정. 
            lean_3p = pd.DataFrame({"lean_3p" : step_data_3p})
            lean_2p = pd.DataFrame({"lean_2p" : step_data_2p})
            lean_1p = pd.DataFrame({"lean_1p" : step_data_1p})    
            
            # 데이터 프레임 합치는 구간. 
            new_df = pd.concat([read_file, lean_3p], axis=1)
            new_df_1 = pd.concat([new_df, lean_2p], axis=1)
            new_df_2 = pd.concat([new_df_1, lean_1p], axis=1)
            
            # Unnamed 0 열 지우기.
            #new_df_2.drop(['Unnamed 0'], axis= 1, inplace=True)
            
            # 엑셀 파일 값으로 저장하는 부분.
            os.chdir(Out_folder) 
            new_df_2.to_excel(f"Box_delta_{week}_{group}_{step}.xlsx")       

## Same열이 있다면, 보이지 않도록 지우기

In [21]:
import os
import pandas as pd

## All_face_lean_5m => 5분 단위로 자른 것을 넣는 부분. 
out_path = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Add_Lean_3p_C\\"
if not os.path.exists(out_path):
    os.makedirs(out_path)

for group in ["C"]:
    for week in ["1W","2W","3W","4W"]:
        # 5분 단위로 나누어서 필요없는 데이터 지우기. 
        for step in ["S1", "S2", "S3"]:
            # "S1_1", "S1_2", "S1_3","S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"
            os.chdir(r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Check_C250\\")
            # D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/box_delta_not_5min/
            # D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/box_delta_5m/
            # r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\Output_A\\"
            
            # New Analysis data file dir
            #r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Check_C250\\"
            
            
            read_file = pd.read_excel(f"Box_delta_{week}_{group}_{step}.xlsx", index_col=0)

            # Unnamed: 0 이라는 이름을 가진 column 지우기. 
            # if 'Unnamed: 0' in read_file.columns:
            #      read_file.drop(['Unnamed: 0'], axis=1, inplace=True)

            # same 열 지우기.
            if 'same' in read_file.columns:
                read_file.drop(['same'], axis=1, inplace=True)
                
            read_file.rename(columns = {
                "box delta": "box_delta_C1",
                "box delta.1": "box_delta_C2",
                "box delta.2": "box_delta_C3",
                "box delta.3": "box_delta_C4"}, inplace=True)    
                            
            read_file.to_excel(out_path + f"Box_delta_{week}_{group}_{step}.xlsx")

## 20분을 5분씩 4개의 파일로 끊어서 저장해보자

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
%matplotlib inline

In [ ]:
import re
import math

for group in ["A","B","C","D","E","F","G"]: # D:\MultiModal\Face_z_score
    os.chdir(f"D:\\MultiModal\\Face_z_score\\{group}\\")
    
    # 파일 불러오기
    file_list = os.listdir(os.getcwd())
    
    # 출력할 파일 불러오기
    output_folder = f"D:\\MultiModal\\Face_z_score\\face_landmark_Split\\{group}\\"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in file_list:
        if file.split('.')[-1] == "xlsx": 
            data_df = pd.read_excel(file)
            data_index = data_df.iloc[:, 1:].columns.to_list()

            file_name = file.split('.')[0]
            #print(file_name.split('_')[4])

            if file_name.split('_')[4] == "S3" or file_name.split('_')[4] == "Baseline":
                data_df.iloc[:, 1:].to_excel(output_folder + file_name + '.xlsx')
            
            else:
                
                num_splits = 4
                
                rows_per_split = len(data_df) // num_splits

                for i in range(num_splits):
                    # 마지막 부분 처리
                    if i == num_splits - 1:
                        split_df = data_df[i * rows_per_split:]
                    else:
                        split_df = data_df[i * rows_per_split:(i + 1) * rows_per_split]
                        
                    split_df.iloc[:, 1:].to_excel(output_folder + f'{file_name}_{i + 1}.xlsx', index=data_index)

                print(f'{num_splits}개의 파일로 나누기 완료.')

## Lean_1p에 대한 값의 합을 구해보자. 

In [ ]:
import os
import pandas as pd

# 합산 값을 보기 위한 output 경로 
output = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Not_5m_lean_1p\\"

data = []

for group in ["A", "B", "C", "D", "E", "F", "G"]:
    for week in ["1W", "2W", "3W", "4W"]:
        for step in ["S1_1","S1_2","S1_3","S1_4","S2_1","S2_2","S2_3","S2_4","S3"]:
            # 현재 디렉토리로 변경 
            os.chdir(r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\All_face_lean_5m\\") 
            read_data = pd.read_excel(f"box_delta_{week}_{group}_{step}.xlsx", index_col=0)
        
            result = read_data["lean_1p"].sum()
            
            data.append(result)
            print(data)

temp_df = pd.DataFrame(data, columns=["SUM_LEAN_1P"])
temp_df.to_excel(output + "LEAN_1P_SUM.xlsx")

## Lean 4p 열을 추가해보자 (same = Lean 4p) 값이랑 똑같음

In [ ]:
import os
import numpy as np
import pandas as pd 


# 파일을 읽기 위한 경로
# read_file = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Modify_time\\Modify_time_5m"

# lean_4p를 추가하기 위해 필요한 output 경로
output = "D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\Add_Lean_4p_C\\"
# D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\Modify_box_size_all\\Modify_time_5m

# 5분으로 나누지 않은 값을 기록한 부분. 
# D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/Modify_box_size_all/

for week in ["1W", "2W", "3W", "4W"]:
    for group in ["C"]:
        for step in ["S1", "S2", "S3"]:
            # ["S1_1", "S1_2", "S1_3", "S2_1", "S2_2", "S2_3", "S3"]:
            # ["S1", "S2", "S3"]:
            
            # 현재 디렉토리로 변경 
            os.chdir(r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\Rename_C\\")
            #os.chdir(r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/Modify_time") 
            # 5m 간격으로 자른 것들 
            # os.chdir(r"D:/MultiModal/Face_z_score/Face_lean_count/backup/All_face_box_zip/Modify_time/Modify_time_5m")
            
            #파일을 읽어옴. 
            read_data = pd.read_excel(f"box_delta_{week}_{group}_{step}.xlsx", index_col=0)
           
            # lean_4p를 저장하기 위한 리스트 선언. 
            step_data_4p = []
        
            # 동시에 기울어진 사람들을 카운트 하는 구간. 
            read_data['same'] = 0 
                    
            for i in range (len(read_data)):
                # box_size 1~4 까지 있는 것들을 세어야 하는 것이므로. 
                #print(read_data.iloc[i, :4])
                val_count = read_data.iloc[i, :4].value_counts()
                # val_count 안에 있는 값이 0 이라면 0으로 초기화
                val_count[0] = 0
                
                #print(read_data.iloc[i, :8])
                #print(read_file.iloc[i, 5])
                
                # 최댓값을 넣어주려는 부분. 
                #print(read_data.iloc[i, 7])
                read_data.iloc[i, 7] = val_count.max()
            
            same = np.array(read_data["same"])

            # 동시에 4명 중 N명이 기울었을 때를 세는 부분 || 모두 list 안에 들어있는 것. 
            step_data_4p = np.where(same>=4,1,0)
            
            # lean_4p에 맞게 데이터 값을 데이터 프레임으로 지정. 
            lean_4p = pd.DataFrame({"lean_4p" : step_data_4p})
            new_df = pd.concat([read_data, lean_4p], axis=1)
    
            if 'same' in new_df.columns:
                new_df.drop(['same'], axis=1, inplace=True) 
            
            # 위치 바꿔주는 부분 넣어줄 것. 
            new_df.insert(4, 'lean_4p', new_df.pop('lean_4p'))    
            
            #엑셀 파일 값으로 저장하는 부분. 
            new_df.to_excel(output + f"box_delta_{week}_{group}_{step}.xlsx")       


## Lean 4p의 합계를 계산해보자

In [ ]:
# Lean_4P 몇 개인지 볼 것. 
output = r"D:\\MultiModal\\Face_z_score\\Face_lean_count\backup\All_face_box_zip\\05_27_box_delta_zip\\Lean_4p_SUM\\"

data = []
for week in ["1W", "2W", "3W", "4W"]:
    for group in ["C"]:
        for step in ["S1", "S2", "S3"]:
            # "S1_1", "S1_2", "S1_3", "S2_1", "S2_2", "S2_3", 
            # "S1", "S2", "S3"
            os.chdir(r"D:\\MultiModal\\Face_z_score\\Face_lean_count\\backup\\All_face_box_zip\\05_27_box_delta_zip\\Add_Lean_4p_A") 
            read_data = pd.read_excel(f"box_delta_{week}_{group}_{step}.xlsx", index_col=0)
        
            result = read_data["lean_4p"].sum()
            
            data.append(result)
            
            #데이터 확인해볼 것 
            # print(data)

temp_df = pd.DataFrame(data, columns=["SUM_LEAN_4P"])
os.chdir(output)
temp_df.to_excel("LEAN_4P_SUM_A.xlsx")